In [1]:
%reload_kedro

2023-03-11 17:47:15,024 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: /Users/guilhermepalazzo/Desktop/dev/git_repositories/crypto_msc_thesis
2023-03-11 17:47:16,339 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-03-11 17:47:17,007 - kedro.extras.extensions.ipython - INFO - ** Kedro project crypto_thesis
2023-03-11 17:47:17,008 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


In [2]:
import pandas as pd

In [3]:
df_xgb_portf_metr = catalog.load("xgboost_portfolio_metrics")
df_xgb_portf_metr.loc[:, "model"] = "xgboost"

df_lstm_portf_metr = catalog.load("lstm_portfolio_metrics")
df_lstm_portf_metr.loc[:, "model"] = "lstm"

df_logreg_portf_metr = catalog.load("logistic_regr_portfolio_metrics")
df_logreg_portf_metr.loc[:, "model"] = "logreg"

df_benchm_metr = catalog.load("benchmark_metrics")
df_benchm_metr.loc[:, "model"] = "benchmark"

2023-03-11 17:47:17,062 - kedro.io.data_catalog - INFO - Loading data from `xgboost_portfolio_metrics` (ParquetDataSet)...
2023-03-11 17:47:17,124 - kedro.io.data_catalog - INFO - Loading data from `lstm_portfolio_metrics` (ParquetDataSet)...
2023-03-11 17:47:17,133 - kedro.io.data_catalog - INFO - Loading data from `logistic_regr_portfolio_metrics` (ParquetDataSet)...
2023-03-11 17:47:17,143 - kedro.io.data_catalog - INFO - Loading data from `benchmark_metrics` (ParquetDataSet)...


In [4]:
final_df = pd.concat([df_xgb_portf_metr, df_lstm_portf_metr, df_logreg_portf_metr, df_benchm_metr])
final_df = final_df.sort_values(by="annual_sharpe", ascending=False)

In [5]:
final_df

,annual_sharpe,profit_factor_pct,nominal_profit,annual_sortino,max_drawdown_pct,consecutive_wins,consecutive_losses,model
0,2.479493,1.408571,13838.688113,4.040618,-0.021300,3,5,lstm
0,0.974147,1.209508,22016.280000,1.387129,-0.155866,10,4,benchmark
0,0.044263,1.006683,-1191.238793,0.062889,-0.037816,5,7,xgboost
0,-2.339782,0.698293,-26671.038251,-2.815370,-0.054896,10,8,logreg


### Operations Profit over Time

In [6]:
# está legado isso!

df_portf_pnl[["close_time", "op_full_profit"]].set_index("close_time").sort_index(). \
                plot(title="Operations' profit over time", 
                     legend=None,
                     xlabel="date",
                     ylabel="profit"
                    )

NameError: name 'df_portf_pnl' is not defined